In [1]:
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import os
import matplotlib.pyplot as plt

# Initialization
annFile = '_annotations.coco.json'
dataDir = 'train'
coco = COCO(annFile)

# Define the classes of interest
filterClasses = ['Basophil', 'Eosinophil', 'Erythroblast', 'Lymphocyte', 'Monocyte', 'Myelocyte', 'Neutrophil', 'Platelet', 'RBC']
catIds = coco.getCatIds(catNms=filterClasses)

# Function to get class name by ID
def getClassName(classID, cats):
    for cat in cats:
        if cat['id'] == classID:
            return cat['name']
    return "None"


# Create directories for saving images and masks
images_dir = "NEW_DATA_V2/Train/images"
masks_dir = "NEW_DATA_V2/Train/masks"
os.makedirs(images_dir, exist_ok=True)
os.makedirs(masks_dir, exist_ok=True)

# Get all images that contain any of the filterClasses
imgIds = coco.getImgIds(catIds=catIds)
print(len(imgIds))

# Process each image
for imgId in imgIds:
    img = coco.loadImgs(imgId)[0]
    annIds = coco.getAnnIds(imgIds=img['id'], catIds=catIds, iscrowd=None)
    anns = coco.loadAnns(annIds)
    
    # Ensure there's at least one annotation before proceeding
    if not anns:
        continue

    # Generate segmentation mask
    mask = np.zeros((img['height'], img['width']), dtype=np.uint8)
    for ann in anns:
        className = getClassName(ann['category_id'], coco.loadCats(catIds))
        pixel_value = filterClasses.index(className) + 1
        mask = np.maximum(coco.annToMask(ann) * pixel_value, mask)
    
    # Save the mask
    mask_filename = os.path.join(masks_dir, f"{img['file_name'].split('.')[0]}_mask.png")
    plt.imsave(mask_filename, mask)

print("All masks have been generated and saved successfully.")


loading annotations into memory...
Done (t=1.66s)
creating index...
index created!
0
All masks have been generated and saved successfully.


In [2]:
from pycocotools.coco import COCO
import numpy as np
import os
import skimage.io as io
import matplotlib.pyplot as plt

# Initialize COCO
annFile = '_annotations.coco.json'
dataDir = 'train'
coco = COCO(annFile)

# Specify directories for saving images and masks
saved_images_dir = 'NEW_DATA_V2/Train/images'
saved_masks_dir = 'NEW_DATA_V2/Train/masks'

# Ensure directories exist
os.makedirs(saved_images_dir, exist_ok=True)
os.makedirs(saved_masks_dir, exist_ok=True)


# Define classes of interest
filterClasses = ['Basophil', 'Eosinophil', 'Erythroblast', 'Lymphocyte', 'Monocyte', 'Myelocyte', 'Neutrophil', 'Platelet', 'RBC']
catIds = coco.getCatIds(catNms=filterClasses)
imgIds = coco.getImgIds(catIds=catIds)

# Function to get class name by ID
def getClassName(classID, cats):
    for cat in cats:
        if cat['id'] == classID:
            return cat['name']
    return "None"

# Iterate over each image ID
for imgId in imgIds:
    img = coco.loadImgs(imgId)[0]
    I = io.imread(f"{dataDir}/{img['file_name']}") / 255.0
    annIds = coco.getAnnIds(imgIds=img['id'], catIds=catIds, iscrowd=None)
    anns = coco.loadAnns(annIds)
    
    # Generate segmentation mask
    mask = np.zeros((img['height'], img['width']), dtype=np.uint8)
    for ann in anns:
        className = getClassName(ann['category_id'], coco.loadCats(catIds))
        if className in filterClasses:
            pixel_value = filterClasses.index(className) + 1
            mask = np.maximum(coco.annToMask(ann) * pixel_value, mask)
    
    # Save image and mask
    img_filename = os.path.join(saved_images_dir, img['file_name'])
    mask_filename = os.path.join(saved_masks_dir, f"{img['file_name'].split('.')[0]}_mask.png")
    io.imsave(img_filename, (I * 255).astype(np.uint8))
    # io.imsave(mask_filename, mask)
     # Save the mask
    
    plt.imsave(mask_filename, mask)

print(f"Completed. Images processed: {len(imgIds)}. Masks saved in '{saved_masks_dir}'.")


loading annotations into memory...
Done (t=1.67s)
creating index...
index created!
Completed. Images processed: 0. Masks saved in 'NEW_DATA_V2/Train/masks'.
